In [2]:
!nvidia-smi 

Sat Jan 23 02:25:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
import numpy as np
print(torch.__version__)
print(np.__version__)

1.7.0+cu101
1.19.5


In [ ]:
!pip install seqeval==0.0.17
!pip install pytorch_crf
!pip install transformers==3.5.1

# Load Data

In [5]:
import numpy as np
import os
import torch.utils.data as data
from sklearn.model_selection import train_test_split
import json
class SentenceGetter:
    def __init__(self, type_data):
        super(SentenceGetter, self).__init__()
        # syllable
        path='/content/drive/MyDrive/Thesis/data/data2/train'
        if (type_data == 'test'):
            path='/content/drive/MyDrive/Thesis/data/data2/test'

        arr = os.listdir(path)

        self.list_sentences = []
        self.list_labels = []
        self.list_pos=[]
        self.list_uppercases = []
        self.list_is_numbers = []

        self.sentences = []
        self.labels = []
        
        
        sentence = ""
        labels = ""
        pos=""
        uppercase=""
        is_number=""
        for i in range(len(arr)):
            path_file = path + "/" + arr[i]

            file_data = open(path_file, encoding='utf-8')

            for line in file_data:

                if line == '\n':
                    sentence = sentence[:-1]
                    labels = labels[:-1]
                    pos=pos[:-1]
                    self.list_sentences.append(sentence)
                    self.list_labels.append(labels)
                    self.list_pos.append(pos)
                    self.list_uppercases.append(uppercase)
                    self.list_is_numbers.append(is_number)
                    sentence = ""
                    labels = ""
                    pos=""
                    uppercase=""
                    is_number=""

                else:
                    words = line.split(" ")
                    sentence = sentence + words[0] + " "
                    pos=pos+words[1]+" "
                    uppercase=uppercase+words[2]+" "
                    is_number=is_number+words[3]+" "
                    labels = labels + words[4].replace("\n", "") + " "
    def data(self,):
      sentences = [[word for word in sentence.split(" ")] for sentence in self.list_sentences]
      labels = [[word for word in sentence.split(" ")] for sentence in self.list_labels]
      poss = [[word for word in sentence.split(" ")] for sentence in self.list_pos] 
      uppercases = [[word for word in sentence.split(" ")] for sentence in self.list_uppercases]
      is_numbers = [[word for word in sentence.split(" ")] for sentence in self.list_is_numbers] 
      return sentences,labels,poss
    



  


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:

# tag_2_int = {w: i for i, w in enumerate(['I-Major_Mode', 'I-duration', 'I-Uni_Name', 'I-Campus_Name', 'B-Person_Name', 'O', 'I-number', 'I-Major_Name', 'B-Scholar_Name', 'I-K_years', 'B-K_years', 'B-datetime', 'B-Docs_Name', 'B-Admission_Type', 'B-Uni_Name', 'B-Dept_Name', 'I-Scholar_Name', 'I-Person_Name', 'I-Docs_Name', 'B-Campus_Name', 'B-Subject_Name', 'I-Subject_Name', 'B-duration', 'I-Dept_Name', 'B-Major_Mode', 'I-datetime', 'B-number', 'B-Major_Name', 'B-location','I-location', 'B-User_PhoneNumber', 'I-Admission_Type', 'I-User_PhoneNumber'])}
# pos_tag_set=['Nb', 'Ny', 'L', 'M', 'Vb', 'C', 'V', 'I', 'P', 'Z', 'Cc', 'CH', 'A', 'R', 'Np', 'N', 'X', 'E', 'T', 'Nc', 'Nu', 'Ni', 'Ab']

# tag_2_int = {w: i for i, w in enumerate(['I-Uni_Name', 'B-Uni_Name', 'I-Person_Name', 'B-Person_Name', 'I-datetime', 'B-datetime', 'I-Subject_Name', 'B-Subject_Name', 'I-duration', 'B-duration', 'I-Scholar_Name', 'B-Scholar_Name', 'I-Major_Mode', 'B-Major_Mode', 'I-K_years', 'B-K_years', 'I-Dept_Name', 'B-Dept_Name', 'I-Campus_Name', 'B-Campus_Name', 'I-Docs_Name', 'B-Docs_Name', 'I-number', 'B-number', 'I-Admission_Type', 'B-Admission_Type', 'I-Major_Name', 'B-Major_Name', 'O'])}
tag_2_int = {w: i for i, w in enumerate(['I-Scholar_Name', 'B-Scholar_Name', 'I-Campus_Name', 'B-Campus_Name', 'I-duration', 'B-duration', 'O', 'I-Admission_Type', 'B-Admission_Type', 'I-Uni_Name', 'B-Uni_Name', 'I-Docs_Name', 'B-Docs_Name', 'I-number', 'B-number', 'I-Major_Mode', 'B-Major_Mode', 'I-K_years', 'B-K_years', 'I-Subject_Name', 'B-Subject_Name', 'I-Person_Name', 'B-Person_Name', 'I-datetime', 'B-datetime', 'I-Dept_Name', 'B-Dept_Name', 'I-Major_Name', 'B-Major_Name'])}
pos_tag_set=['Nb', 'Ny', 'L', 'M', 'Vb', 'C', 'V', 'I', 'P', 'Z', 'Cc', 'CH', 'A', 'R', 'Np', 'N', 'X', 'E', 'T', 'Nc', 'Nu', 'Ni', 'Ab']
# pos_tag_set=['Nc', 'N', 'C', 'I', 'CH', 'Ni', 'M', 'P', 'L', 'Z', 'Ab', 'Vb', 'Cc', 'A', 'E', 'Ny', 'V', 'Nu', 'T', 'Np', 'X', 'Nb', 'R']

pos2id={ pos:i   for i,pos in enumerate(pos_tag_set)}
with open('/content/drive/MyDrive/Thesis/data/tags/charindex.json','r',encoding='utf-8') as fr:
  char2id=json.load(fr)

# Create Dataloader

In [7]:
import torch
import argparse
from torch.utils.data import DataLoader,TensorDataset,RandomSampler,SequentialSampler
from keras.preprocessing.sequence import pad_sequences



from transformers import AutoModel,AutoTokenizer
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
 


## Config Maxlen_sentence, batch_size

In [8]:
MAX_LEN = 80
bs = 128
MAX_LEN_CHAR=20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


## Load Tokenizer

In [9]:
tokenizer=AutoTokenizer.from_pretrained('vinai/phobert-base')

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


## Process word,label to subword and subword's label

In [10]:
def tokenize_and_preserve_labels(sentence=None, text_labels=None,list_pos=None):
  if text_labels is not None:
    tokenized_sentence = ['<s>']
    labels = ['O']
    poss=['X']
    for word, label,pos in zip(sentence, text_labels,list_pos):

        # tách từ thành subword theo bert tokenizer và đếm số từ tách subword
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)
        tokenized_sentence.extend(tokenized_word)

        # thêm label theo subword
        labels.extend([label] * n_subwords)
        poss.extend([pos]*n_subwords)
    tokenized_sentence.append('</s>')
    labels.append('O')
    poss.append('X')
    return tokenized_sentence, labels, poss
  else:
    tokenized_sentence = ['<s>']
    
    poss=['X']
    for word, pos in zip(sentence,list_pos):

        # tách từ thành subword theo bert tokenizer và đếm số từ tách subword
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)
        tokenized_sentence.extend(tokenized_word)

        poss.extend([pos]*n_subwords)
    tokenized_sentence.append('</s>')
   
    poss.append('X')
    return tokenized_sentence, poss


In [11]:
def Create_data(data):

    sentences, labels, poss=data.data()
    tokenized_texts_and_labels = [
        tokenize_and_preserve_labels(sent, labs,pos)
        for sent, labs,pos in zip(sentences, labels,poss)
        
    ]
    tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
    labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]
    pos_tags=[token_label_pair[2] for token_label_pair in tokenized_texts_and_labels]

    return tokenized_texts,pos_tags,labels



## Load Character

In [12]:
def Character(tokenized_texts,char2id,MAX_LEN,MAX_LEN_CHAR):
    X_char=[]
    char_len=[]
    word_len=[]
    for sentence in tokenized_texts:
      word_len.append(len(sentence))
      sent_seq=[]
      
      sent_char_len=[]
      sentence=pad_sequences([sentence],maxlen=MAX_LEN,value='<pad>',padding='post',dtype=object)[0]
    
      for word in sentence:
        word_seq=[]
        sent_char_len.append(len(word))
        for j in range(MAX_LEN_CHAR):
          try:
            char=word[j]
            
            if char not in char2id:
              word_seq.append(char2id.get('UNK'))
            else:
              word_seq.append(char2id.get(char))
          except:
            word_seq.append(char2id.get('PAD'))
        sent_seq.append(word_seq)
      X_char.append(sent_seq)
      char_len.append(sent_char_len)
    return X_char,char_len  

## Create data train and validation

In [13]:
def Create_DataLoader(tokenizer,tag_2_int,pos2id,char2id,MAX_LEN,MAX_LEN_CHAR,bs,data):

    tokenized_texts,pos_tags,labels=Create_data(data)

    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                              maxlen=MAX_LEN, dtype="long", value=1.0,
                              truncating="post", padding="post")
    tags = pad_sequences([[tag_2_int.get(l) for l in lab] for lab in labels],
                        maxlen=MAX_LEN, value=tag_2_int["O"], padding="post",
                        dtype="long", truncating="post")
    attention_masks = [[float(i != 1.0) for i in ii] for ii in input_ids]

    list_pos=pad_sequences([[pos2id.get(pos) for pos in posi] for posi in pos_tags],
                          maxlen=MAX_LEN,dtype="long",value=pos2id.get('X'),
                          padding="post",truncating="post")
    char,char_len=Character(tokenized_texts,char2id=char2id,MAX_LEN=MAX_LEN,MAX_LEN_CHAR=MAX_LEN_CHAR)


    inputs=torch.tensor(input_ids)
    labels=torch.tensor(tags)
    attention_masks=torch.tensor(attention_masks)
    pos_tags=torch.tensor(list_pos)
    char=torch.tensor(char)
    char_len=torch.tensor(char_len)

    # return inputs.shape,labels.shape,attention_masks.shape,pos_tags.shape,char.shape,char_len.shape

    tensor_data=TensorDataset(inputs,pos_tags.view(pos_tags.shape[0],pos_tags.shape[1]),attention_masks,char,char_len,labels)
    sampler=RandomSampler(tensor_data)
    dataloader=DataLoader(tensor_data,batch_size=bs,sampler=sampler)

    return dataloader




# Model

In [14]:
from transformers import *
import torch.nn as nn
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from torchcrf import CRF
from torch.optim import SGD,Adam
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

## Character representation

### Char LSTM

In [15]:
class CharLSTM(nn.Module):

  def __init__(self,vocab_size,hidden_dim,embedding_dim,dropout=0.1):
    super(CharLSTM, self).__init__()
    self.hidden_dim=hidden_dim
    self.embedding_dim=embedding_dim
    self.embedding=nn.Embedding(num_embeddings=vocab_size,embedding_dim=embedding_dim)
    self.char_bilstm=nn.LSTM(input_size=self.embedding_dim,hidden_size=self.hidden_dim,bidirectional=True,batch_first=True, num_layers=1)
    self.dropout=nn.Dropout(dropout)
    self.embedding.weight.data.copy_ = torch.from_numpy(self.random_embedding(vocab_size, embedding_dim))
   

  def random_embedding(self, vocab_size, embedding_dim):
    pretrain_emb = np.empty([vocab_size, embedding_dim])
    scale = np.sqrt(3.0 / embedding_dim)
    for index in range(vocab_size):
      pretrain_emb[index, :] = np.random.uniform(-scale, scale, [1, embedding_dim])
    return pretrain_emb

  
  def forward(self,char_seq,char_seq_len):
    batch_size=char_seq.size(0)
    sent_len=char_seq.size(1)
    
    char_seq=char_seq.view(batch_size*sent_len,char_seq.size(2))
    char_seq_len=char_seq_len.view(batch_size*sent_len,1).transpose(0,1).squeeze()
    text_emb = self.embedding(char_seq)
    packed_input = pack_padded_sequence(text_emb, char_seq_len.cpu(), batch_first=True, enforce_sorted=False)

    packed_output,_=self.char_bilstm(packed_input)
    output, _ = pad_packed_sequence(packed_output, batch_first=True)    
    out_forward = output[range(len(output)), char_seq_len - 1, :self.hidden_dim]
    out_reverse = output[:, 0, self.hidden_dim:]
    out_reduced = torch.cat((out_forward, out_reverse), -1)
    out=self.dropout(out_reduced)
    out=out.view(batch_size,sent_len,-1)
    return out
        




### Char CNN

In [16]:
class CharCNN(nn.Module):
    def __init__(self, hidden_dim, embedding_dim, vocab_size, weights=None, dropout=0.1):
        super(CharCNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        if weights is not None:
            self.embeddings.weight = nn.Parameter(weights, requires_grad=False)
        else:
            self.embeddings.weight.data.copy_ = torch.from_numpy(self.random_embedding(vocab_size, hidden_dim))

        self.kernels = [3, 4]
        cnns = []

        for k in self.kernels:
            seq = nn.Sequential(
                nn.Conv1d(embedding_dim, self.hidden_dim, kernel_size=k, padding=1),
                nn.Tanh()
            )
            cnns.append(seq)
        self.cnns = nn.ModuleList(cnns)
        self.dropout = nn.Dropout(dropout)

    def random_embedding(self, vocab_size, embedding_dim):
        pretrain_emb = np.empty([vocab_size, embedding_dim])
        scale = np.sqrt(3.0 / embedding_dim)
        for index in range(vocab_size):
            pretrain_emb[index, :] = np.random.uniform(-scale, scale, [1, embedding_dim])
        return pretrain_emb

    def forward(self, x):
        seq_len = x.size(1)
        batch_size = x.size(0)
        x = x.view(batch_size * seq_len, x.size(2))
        x = self.embeddings(x)
        x = torch.transpose(x, 2, 1)
        tmp = [cnn(x) for cnn in self.cnns]
        char_cnn = []
        for item in tmp:
            char_cnn.append(F.max_pool1d(item, item.size(2)))

        char_cnn = torch.cat(char_cnn, dim=1)
        char_out = char_cnn.view(batch_size, seq_len, -1)
        char_out = self.dropout(char_out)
        return char_out


## Pos embedding

In [17]:
class PoSEmbedding(nn.Module):
    """Part-of-Speech embeddings
    Trained with part-of-speech tag, the embedding layer will be the PoS embeddings
    """
    def __init__(self, vocab_size, embedding_size, num_entities,dropout=0.1):
        super(PoSEmbedding,self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.fc = nn.Linear(embedding_size, num_entities)
        self.embedding.weight.data.copy_ = torch.from_numpy(self.random_embedding(vocab_size, embedding_size))
        self.dropout=nn.Dropout(dropout)
      
    def random_embedding(self, vocab_size, embedding_dim):
        pretrain_emb = np.empty([vocab_size, embedding_dim])
        scale = np.sqrt(3.0 / embedding_dim)
        for index in range(vocab_size):
            pretrain_emb[index, :] = np.random.uniform(-scale, scale, [1, embedding_dim])
        return pretrain_emb
    def forward(self, x):
        
        # seq_len = x.size(1)
        # batch_size = x.size(0)
        # # print(x.shape)
        # x = x.view(batch_size * seq_len, x.size(2))
        x = self.embedding(x)
        x = self.fc(x)  # (B, num_entities)
        # x=x.view(batch_size,seq_len,-1)
        x=self.dropout(x)
        return x

## Main model

In [18]:
class SeqLabel(nn.Module):
   
   def __init__(self,char_emb_dim=100,char_seq_feature='lstm',char_vocab_size=108,char_hidden_dim=256,pos_vocab_size=23,pos_emb_dim=22,pos_hidden_dim=50,hidden_dim=768,lstm_layer=2,dropout=0.05,num_labels=None):
  #  def __init__(self,char_emb_dim=30,char_seq_feature='lstm',char_vocab_size=108,char_hidden_dim=50,pos_vocab_size=23,pos_emb_dim=22,pos_hidden_dim=50,hidden_dim=200,lstm_layer=2,dropout=0.05,num_labels=None):
       super(SeqLabel,self).__init__()
       config=AutoConfig.from_pretrained('vinai/phobert-base')
       self.num_labels = num_labels
       self.char_seq_feature=char_seq_feature
       if char_seq_feature=='cnn':
         self.char_feature=CharCNN(hidden_dim=char_hidden_dim, embedding_dim=char_emb_dim, vocab_size=char_vocab_size)

       else:
         self.char_feature=CharLSTM(vocab_size=char_vocab_size,hidden_dim=char_hidden_dim,embedding_dim=char_emb_dim)

       self.pos_feature=PoSEmbedding(vocab_size=pos_vocab_size, embedding_size=pos_emb_dim, num_entities=pos_hidden_dim)
     
       self.bert = AutoModel.from_pretrained('vinai/phobert-base')
      
       
       self.bilstm=nn.LSTM(bidirectional=True,input_size=config.hidden_size*4+2*char_hidden_dim+pos_hidden_dim,hidden_size=hidden_dim//2,num_layers=2,batch_first=True)
      
       self.classifier = nn.Linear(hidden_dim, self.num_labels)
      
       self.dropout=nn.Dropout(dropout)
      
       self.crf = CRF(num_tags=self.num_labels, batch_first=True)
       
      #  for param in self.bert.parameters():
      #    param.requires_grad = False
       
   



   def forward(
        self,
        input_ids=None,
        char_id=None,
        char_len=None,
        pos_id=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):

       outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_hidden_states=True
        )
       if self.char_seq_feature=='lstm':
        char_output=self.char_feature(char_id,char_len)
       else:
         char_output=self.char_feature(char_id)
       
       
       pos_output=self.pos_feature(pos_id)
       
       output=torch.cat((outputs[2][-1],outputs[2][-2],outputs[2][-3],outputs[2][-4],char_output,pos_output),dim=-1)
      

       logits,_=self.bilstm(output)
  
       logits = self.classifier(logits)
     
       logtis=self.dropout(logits)
 
       if labels is not None:
          attention_mask=attention_mask.type(torch.uint8)
          
          loss,output=-self.crf(logits,tags=labels,mask=attention_mask,reduction='mean'),self.crf.decode(logits)
          return loss,output
       else:
          output=self.crf.decode(logits)
          return output


## Early Stopping

In [19]:
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score > self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

## Train

In [20]:
train=SentenceGetter('train')
test=SentenceGetter('test')

train_dataloader=Create_DataLoader(tokenizer=tokenizer,tag_2_int=tag_2_int,char2id=char2id,pos2id=pos2id,MAX_LEN=MAX_LEN,MAX_LEN_CHAR=MAX_LEN_CHAR,data=train,bs=bs)
val_dataloader=Create_DataLoader(tokenizer=tokenizer,tag_2_int=tag_2_int,char2id=char2id,pos2id=pos2id,MAX_LEN=MAX_LEN,MAX_LEN_CHAR=MAX_LEN_CHAR,bs=bs,data=test)

In [21]:
model=SeqLabel(char_seq_feature='cnn',char_hidden_dim=200,pos_hidden_dim=100,num_labels=29)
# model=SeqLabel(char_seq_feature='cnn',num_labels=29)
model.cuda();

## Optimizer

In [22]:

param_optimizer=list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
    'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
    'weight_decay_rate': 0.0}
]

optimizer = AdamW(
    optimizer_grouped_parameters ,eps=1e-8,
    lr=1e-4
    ,weight_decay=0.01
)
# optimizer = AdamW(
#    param_optimizer,
#    eps=1e-8,
#    lr=5e-3,weight_decay=0.01
    
# )
# param_optimizer=filter(lambda p: p.requires_grad, model.parameters())


## Config training

In [23]:

epochs = 100
max_grad_norm = 1.0
early_stopping = EarlyStopping(patience=20, verbose=True)
total_steps = len(train_dataloader)*epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [24]:
from tqdm.notebook import trange
from seqeval.metrics import f1_score,classification_report

# tag_values=np.array(['I-Uni_Name', 'B-Uni_Name', 'I-Person_Name', 'B-Person_Name', 'I-datetime', 'B-datetime', 'I-Subject_Name', 'B-Subject_Name', 'I-duration', 'B-duration', 'I-Scholar_Name', 'B-Scholar_Name', 'I-Major_Mode', 'B-Major_Mode', 'I-K_years', 'B-K_years', 'I-Dept_Name', 'B-Dept_Name', 'I-Campus_Name', 'B-Campus_Name', 'I-Docs_Name', 'B-Docs_Name', 'I-number', 'B-number', 'I-Admission_Type', 'B-Admission_Type', 'I-Major_Name', 'B-Major_Name', 'O'])
tag_values=np.array(['I-Scholar_Name', 'B-Scholar_Name', 'I-Campus_Name', 'B-Campus_Name', 'I-duration', 'B-duration', 'O', 'I-Admission_Type', 'B-Admission_Type', 'I-Uni_Name', 'B-Uni_Name', 'I-Docs_Name', 'B-Docs_Name', 'I-number', 'B-number', 'I-Major_Mode', 'B-Major_Mode', 'I-K_years', 'B-K_years', 'I-Subject_Name', 'B-Subject_Name', 'I-Person_Name', 'B-Person_Name', 'I-datetime', 'B-datetime', 'I-Dept_Name', 'B-Dept_Name', 'I-Major_Name', 'B-Major_Name'])
print(len(tag_values))

29


In [ ]:
for epoch in trange(epochs, desc='Epoch'):
  
  # TRAIN
  
  model.train()
  total_loss = 0
  for batch in train_dataloader:
    # add to GPU
    batch = tuple(t.to(device) for t in batch)
    tr_inputs,tr_pos_tags,tr_attention_masks,tr_char,tr_char_len,tr_labels = batch


    
    # clear gradient after each backward
    model.zero_grad()

    # get loss and score
    output = model(input_ids=tr_inputs,char_id=tr_char,char_len=tr_char_len,pos_id=tr_pos_tags,attention_mask=tr_attention_masks,labels=tr_labels)
    
    loss = output[0]
    loss.backward()

    total_loss += loss.item()
    
    # norm gradient
    torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
    
    # update parameter
    optimizer.step()
    # update learning rate
    scheduler.step()
  
  print('Average train loss: {}'.format(total_loss/len(train_dataloader)))
  val_loss=0
 
  # VALIDATION
  model.eval()

  pred_labels_ids, true_labels_ids, val_ids_sent = [], [], []
  for batch in val_dataloader:
    batch = tuple(t.to(device) for t in batch)
    val_inputs,val_pos_tags,val_attention_masks,val_char,val_char_len,val_labels = batch

    # return (score)
    with torch.no_grad():
      output_val = model(input_ids=val_inputs,char_id=val_char,char_len=val_char_len,pos_id=val_pos_tags,attention_mask=val_attention_masks,labels=val_labels)
    
    # move to CPU
    loss=output_val[0]
    val_loss+=loss.item()
    val_ids_sent.extend(val_inputs)
    logit = torch.tensor(output_val[1]).detach().cpu().numpy()
    label_ids = val_labels.to('cpu').numpy()

    pred_labels_ids.extend(logit)
    true_labels_ids.extend(label_ids)
  val_loss=val_loss/len(val_dataloader)
  print('Average val loss: {}'.format(val_loss))
  
 
  # dont use PAD labels to compute f1 score
  pred_labels = [tag_values[pred_indx] for pred, true in zip(pred_labels_ids, true_labels_ids)
                                      for pred_indx, true_indx in zip(pred, true) if tag_values[true_indx] != 'O']
  true_labels = [tag_values[indx] for true in true_labels_ids
                                      for indx in true if tag_values[indx] != 'O']
  f1=f1_score(true_labels,pred_labels)
  print('{}. Validation F1-score: {}\n'.format(epoch+1,f1 ))  
  early_stopping(f1, model)
  if early_stopping.early_stop:
            print("Early stopping")
            break
model.load_state_dict(torch.load('checkpoint.pt'))
  

## Test

Test each sentence

In [27]:
test=SentenceGetter('test')
test_sentences, test_labels, test_pos_tags=test.data()

In [26]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Thesis/bert-cnn-fold1-2.pt'))

<All keys matched successfully>

In [28]:
predict=[]

with open('/content/drive/MyDrive/output.txt','w',encoding='utf-8') as out:
  for k in range(0,len(test_sentences)):
    test_sentence=test_sentences[k]
    test_label=test_labels[k]
    test_pos_tag=test_pos_tags[k]
    
    sent_ids,test_pos_tag=tokenize_and_preserve_labels(sentence=test_sentence,list_pos=test_pos_tag)
    
    # tokenized_sentence = pad_sequences([tokenizer.convert_tokens_to_ids(sent_ids)], maxlen=MAX_LEN, dtype="long",
    #                                         value=1.0, truncating="post", padding="post")
    # input_ids = torch.tensor(tokenized_sentence,dtype=torch.long).to(device)

    # test_pos_id = pad_sequences([[pos2id.get(pos) for pos in test_pos_tag]],
    #                                   maxlen=MAX_LEN, dtype="long", value=pos2id.get('X'),
    #                                   padding="post", truncating="post")

    input_ids=torch.tensor([tokenizer.convert_tokens_to_ids(sent_ids)],dtype=torch.long).to(device)
    test_pos_id=[[pos2id.get(pos) for pos in test_pos_tag]]
    test_pos_id = torch.tensor(test_pos_id,dtype=torch.long).to(device)



    mask = [[float(m != 1) for m in val] for val in input_ids]
    mask = torch.tensor(mask).to(device)

    test_char=[]
    # sentence=pad_sequences([sent_ids],maxlen=MAX_LEN,value='<pad>',padding='post',dtype=object)[0]
    sentence=sent_ids
    test_char_len=[]
    for word in sentence:
      word_seq=[]
      test_char_len.append(len(word))
      for j in range(MAX_LEN_CHAR):
        try:
          char=word[j]
        
          if char not in char2id:
            word_seq.append(char2id.get('UNK'))
          else:
            word_seq.append(char2id.get(char))
        except:
          word_seq.append(char2id.get('PAD'))
      test_char.append(word_seq)
    test_char=[test_char]

    test_char=torch.tensor(test_char).to(device)
    test_char_len=torch.tensor(test_char_len).to(device)
    
    


    with torch.no_grad():
      output = model(input_ids=input_ids,char_id=test_char,char_len=test_char_len,pos_id=test_pos_id,attention_mask=mask)
    label_ids = torch.tensor(output[0]).cpu().numpy()
    subword_label = list(zip([tag_values[label] for label in label_ids], sent_ids))
    sentence = []
    predict_labels = []
    word = ''
    for label, sword in subword_label:
      if '@@' not in sword:
        word += sword
        sentence.append(word)
        predict_labels.append(label)
        word = ''
        continue
      word += sword.replace('@@', '')
    predict_labels=predict_labels[1:-1]
    predict.append(predict_labels)
    for i in range(len(test_sentence)):
      out.write("{} {} {}\n".format(test_sentence[i],test_label[i],predict_labels[i]))
    out.write("\n")

In [39]:
with open('/content/drive/MyDrive/Thesis/data/fail.txt','w',encoding='utf-8') as out:
  for k in range(0,len(test_sentences)):
    test_sentence=test_sentences[k]
    test_label = test_labels[k]
    predicts = predict[k]
    if test_label != predicts:
      for i in range(len(test_sentence)):
          out.write("{} {} {}\n".format(test_sentence[i],test_label[i],predicts[i]))
      out.write("\n")

In [35]:
t = [0,1,3]
t2 = [0,1,2]
t == t2

False

In [ ]:
print(classification_report(test_labels,predict,digits=4))

                precision    recall  f1-score   support

Admission_Type     0.6667    0.8667    0.7536        30
   Campus_Name     0.8750    1.0000    0.9333         7
     Dept_Name     1.0000    1.0000    1.0000         3
     Docs_Name     0.7391    0.8095    0.7727        42
       K_years     0.8000    0.6667    0.7273         6
    Major_Mode     0.7679    0.6615    0.7107        65
    Major_Name     0.8981    0.9216    0.9097       153
   Person_Name     1.0000    0.4000    0.5714         5
  Scholar_Name     0.8947    0.8500    0.8718        40
  Subject_Name     0.8621    0.8333    0.8475        30
      Uni_Name     0.8090    0.8656    0.8364       186
      datetime     0.9326    0.9222    0.9274        90
      duration     0.8000    0.7273    0.7619        11
        number     0.8293    0.9189    0.8718        37

     micro avg     0.8380    0.8582    0.8479       705
     macro avg     0.8482    0.8174    0.8211       705
  weighted avg     0.8409    0.8582    0.8470 

In [ ]:
print(classification_report(test_labels,predict,digits=4))

                precision    recall  f1-score   support

Admission_Type     0.6087    0.9333    0.7368        30
   Campus_Name     0.7000    1.0000    0.8235         7
     Dept_Name     1.0000    1.0000    1.0000         3
     Docs_Name     0.7609    0.8333    0.7955        42
       K_years     0.7500    0.5000    0.6000         6
    Major_Mode     0.7193    0.6308    0.6721        65
    Major_Name     0.8917    0.9150    0.9032       153
   Person_Name     1.0000    0.8000    0.8889         5
  Scholar_Name     0.7955    0.8750    0.8333        40
  Subject_Name     0.6897    0.6667    0.6780        30
      Uni_Name     0.8366    0.9086    0.8711       186
      datetime     0.9053    0.9556    0.9297        90
      duration     0.7500    0.8182    0.7826        11
        number     0.9143    0.8649    0.8889        37

     micro avg     0.8226    0.8681    0.8447       705
     macro avg     0.8087    0.8358    0.8146       705
  weighted avg     0.8262    0.8681    0.8441 

## Save model

In [ ]:
del model
del early_stopping

NameError: ignored

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Thesis/bert-cnn-fold1-2.pt")